<a href="https://colab.research.google.com/github/Mateo289/TAM/blob/main/Parciales/Parcial_1_Punto_3_(Dashboard).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok
!pip install streamlit

In [ ]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')
print("Google Drive montado con éxito.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado con éxito.


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

# --- Configuración de la página Streamlit ---
st.set_page_config(page_title="Comparación de Modelos de Regresión", layout="wide")

# IMPORTANTE: Define las rutas base de tus archivos en Google Drive
DRIVE_FILES_BASE_PATH_1 = '/content/drive/MyDrive/MODELOS_TAM/TEST_DATA' # Ruta para Xtest y ytest
DRIVE_FILES_BASE_PATH_3 = '/content/drive/MyDrive/MODELOS_TAM/RESULTS_DATA' # Ruta para los resultados y predicciones precalculados

# --- Define el tipo de búsqueda que quieres cargar por defecto en el dashboard ---
# Puedes cambiar esto a "GridSearch" si prefieres ver esos resultados.
DEFAULT_SEARCH_TYPE_TO_LOAD = "RandomizedSearch"

# --- Funciones de Carga con Caché ---

@st.cache_data
def load_test_data(base_path):
    """Carga los datos de test (X y y) desde archivos CSV en Drive."""
    try:
        X_path = os.path.join(base_path, 'Xtest_for_streamlit.csv')
        y_path = os.path.join(base_path, 'ytest_for_streamlit.csv')

        X_test = pd.read_csv(X_path)
        y_test = pd.read_csv(y_path).squeeze() # .squeeze() si y_test es una Serie de una columna
        st.sidebar.success("Datos de test (Xtest, ytest) cargados.")
        return X_test, y_test
    except FileNotFoundError:
        st.error(f"Archivos de test (Xtest, ytest) no encontrados en '{base_path}'. Son necesarios para las gráficas y exploración de datos.")
        return pd.DataFrame(), pd.Series()
    except Exception as e:
        st.error(f"Error al cargar los datos de test: {e}")
        return pd.DataFrame(), pd.Series()

@st.cache_data
def load_results_data(base_path, search_type):
    """Carga los resultados de métricas (final_test_results y cv_grid_results) con una etiqueta de búsqueda."""
    try:
        final_results_df = pd.read_csv(os.path.join(base_path, f'final_test_results_{search_type}.csv'), index_col=0)
        cv_results_df = pd.read_csv(os.path.join(base_path, f'cv_grid_results_{search_type}.csv'))
        st.sidebar.success(f"Resultados de métricas ({search_type}) cargados.")
        return final_results_df, cv_results_df
    except FileNotFoundError:
        st.error(f"Archivos de resultados ({search_type}) no encontrados en '{base_path}'. Asegúrate de haberlos guardado con esta etiqueta.")
        return pd.DataFrame(), pd.DataFrame()
    except Exception as e:
        st.error(f"Error al cargar resultados de métricas ({search_type}): {e}")
        return pd.DataFrame(), pd.DataFrame()

@st.cache_data
def load_predictions_data(base_path, search_type):
    """Carga el diccionario de predicciones (y_preds_for_plot) con una etiqueta de búsqueda."""
    try:
        y_preds_pickle_path = os.path.join(base_path, f'y_preds_for_plot_{search_type}.pkl')
        with open(y_preds_pickle_path, 'rb') as f:
            y_preds_for_plot = pickle.load(f)
        st.sidebar.success(f"Predicciones ({search_type}) cargadas.")
        return y_preds_for_plot
    except FileNotFoundError:
        st.error(f"Archivo de predicciones (y_preds_for_plot_{search_type}.pkl) no encontrado en '{base_path}'. Asegúrate de haberlo guardado con esta etiqueta.")
        return {}
    except Exception as e:
        st.error(f"Error al cargar predicciones ({search_type}): {e}")
        return {}

# --- Título del Dashboard ---
st.title("Comparación de Modelos de Regresión para Precios de Viviendas")
st.markdown("Este dashboard visualiza el rendimiento de varios modelos de regresión optimizados.")

# --- Carga de Datos y Resultados Precalculados ---
Xtest_st, ytest_st = load_test_data(DRIVE_FILES_BASE_PATH_1)
final_test_results_df_st, cv_grid_results_df_st = load_results_data(DRIVE_FILES_BASE_PATH_3, DEFAULT_SEARCH_TYPE_TO_LOAD)
y_preds_for_plot_st = load_predictions_data(DRIVE_FILES_BASE_PATH_3, DEFAULT_SEARCH_TYPE_TO_LOAD)


# --- Verificar si se cargaron todos los datos y resultados necesarios ---
if Xtest_st.empty or ytest_st.empty or final_test_results_df_st.empty or not y_preds_for_plot_st:
    st.warning("No se pudieron cargar todos los datos o resultados necesarios para la visualización. Por favor, verifica las rutas de Drive y que los archivos CSV/PKL existan.")
    st.stop() # Detiene la ejecución si no hay datos o resultados

# --- Pestañas del Dashboard ---
tab1, tab2, tab3 = st.tabs(["Rendimiento del Modelo", "Gráficas de Predicción", "Exploración de Datos (Test)"])

with tab1:
    st.header(f"Rendimiento Comparativo de Modelos en el Conjunto de Test ({DEFAULT_SEARCH_TYPE_TO_LOAD} Resultados)")
    # Formateo numérico robusto:
    numeric_cols_format_final = {}
    for col in final_test_results_df_st.columns:
        if pd.api.types.is_numeric_dtype(final_test_results_df_st[col]):
            numeric_cols_format_final[col] = "{:.3f}"
    st.dataframe(final_test_results_df_st.style.format(numeric_cols_format_final))

    st.subheader(f"Rendimiento en Cross-Validation (Métricas del Mejor Estimador - {DEFAULT_SEARCH_TYPE_TO_LOAD} Resultados)")
    # Formateo numérico robusto para CV:
    numeric_cols_format_cv = {}
    for col in cv_grid_results_df_st.columns:
        if pd.api.types.is_numeric_dtype(cv_grid_results_df_st[col]):
            numeric_cols_format_cv[col] = "{:.3f}"
        # Puedes añadir una excepción para la columna 'Model' si es una cadena y no quieres formatearla
        elif col == 'Model' or col.startswith('Param_'): # Los parámetros también pueden ser strings
            numeric_cols_format_cv[col] = "{}" # Formato genérico para no numéricos
    st.dataframe(cv_grid_results_df_st.style.format(numeric_cols_format_cv))


with tab2:
    st.header("Gráficas de Valores Reales vs. Predichos")

    # Colores para las gráficas
    colors = {
        "LinearRegression": "blue", "Lasso": "red", "ElasticNet": "green",
        "KernelRidge": "purple", "SGDRegressor": "orange", "BayesianRidge": "brown",
        "RandomForestRegressor": "darkgreen", "SVR": "darkred", "GaussianProcessRegressor": "darkcyan"
    }

    selected_model_for_plot = st.selectbox(
        "Selecciona un modelo para ver su gráfica de dispersión:",
        options=list(y_preds_for_plot_st.keys())
    )

    if selected_model_for_plot and selected_model_for_plot in y_preds_for_plot_st:
        y_pred_current = y_preds_for_plot_st[selected_model_for_plot]

        fig = px.scatter(
            x=ytest_st, y=y_pred_current,
            labels={'x': 'Valores Reales (ytest)', 'y': f'Valores Predichos ({selected_model_for_plot})'},
            title=f'{selected_model_for_plot}: Valores Reales vs. Predichos en Test Set',
            color_discrete_sequence=[colors.get(selected_model_for_plot, 'gray')]
        )

        # Añadir la línea de predicción ideal (y=x)
        min_val = min(ytest_st.min(), y_pred_current.min())
        max_val = max(ytest_st.max(), y_pred_current.max())
        fig.add_shape(type="line", x0=min_val, y0=min_val, x1=max_val, y1=max_val,
                      line=dict(color="black", width=2, dash="dash"),
                      name='Predicción Ideal (y=x)')

        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("Selecciona un modelo válido para visualizar su gráfica.")

with tab3:
    st.header("Exploración del Conjunto de Datos de Test")
    if not Xtest_st.empty:
        st.subheader("Muestra de Datos de Características (Xtest)")
        st.dataframe(Xtest_st.head())

        st.subheader("Estadísticas Descriptivas de Características (Xtest)")
        st.dataframe(Xtest_st.describe().round(2))

        st.subheader("Distribución de la Variable Objetivo (ytest)")
        fig_ytest_hist = px.histogram(ytest_st, title='Distribución de Valores Reales (ytest)')
        st.plotly_chart(fig_ytest_hist, use_container_width=True)

        st.subheader("Gráfico Interactivo de Columnas (Xtest)")
        col_to_plot = st.selectbox(
            "Selecciona una columna para graficar:",
            options=Xtest_st.columns,
            index=0 if not Xtest_st.empty else 0
        )
        if col_to_plot:
            if pd.api.types.is_numeric_dtype(Xtest_st[col_to_plot]):
                fig_col = px.histogram(Xtest_st, x=col_to_plot, title=f'Distribución de {col_to_plot}')
            else:
                fig_col = px.bar(Xtest_st[col_to_plot].value_counts().reset_index(),
                                 x='index', y=col_to_plot, title=f'Conteo de {col_to_plot}')
            st.plotly_chart(fig_col, use_container_width=True)
    else:
        st.warning("No hay datos de test disponibles para explorar.")

Overwriting app.py


In [ ]:
# Celda nueva: Detener procesos de ngrok y liberar el puerto
print("Deteniendo cualquier proceso de ngrok existente...")
!killall ngrok > /dev/null 2>&1 # Intenta matar todos los procesos de ngrok

print("Liberando el puerto 8501 si está en uso...")
# Encuentra y mata cualquier proceso que esté usando el puerto 8501
!fuser -k 8501/tcp > /dev/null 2>&1 || true

print("Limpieza completada. Ahora puedes intentar iniciar Streamlit de nuevo.")

Deteniendo cualquier proceso de ngrok existente...
Liberando el puerto 8501 si está en uso...
Limpieza completada. Ahora puedes intentar iniciar Streamlit de nuevo.


In [ ]:
from pyngrok import ngrok
import threading
import time

# Opcional: Autentica ngrok si tienes un token para evitar límites de tiempo
ngrok.set_auth_token("2xbtLOEkytIQJvweL5UHGOuUCTv_3vr83C6BkLdGb8KxmHcXJ") # <--- Descomenta y reemplaza si tienes un token

# Inicia Streamlit en un hilo de fondo
def run_streamlit():
    # Deshabilitamos la protección CSRF para Colab/ngrok para evitar problemas de conexión.
    # También deshabilitamos CORS para permitir el acceso desde el túnel.
    !streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false

print("Iniciando Streamlit...")
threading.Thread(target=run_streamlit).start()

# Espera un momento para que Streamlit se inicie completamente
time.sleep(10) # Aumentado a 10 segundos para dar más tiempo a que Streamlit y los modelos se carguen

# Configura ngrok para exponer el puerto de Streamlit (8501)
try:
    public_url = ngrok.connect(addr="8501", proto="http")
    print(f"Tu aplicación Streamlit está disponible en: {public_url}")
    print("¡Haz clic en el enlace de arriba para abrir tu dashboard!")
except Exception as e:
    print(f"Error al conectar con ngrok: {e}")
    print("Asegúrate de que Streamlit se está ejecutando correctamente y que el puerto 8501 está libre.")
    print("Si tienes un token de ngrok, asegúrate de haberlo configurado.")

Iniciando Streamlit...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.9.112:8501

Tu aplicación Streamlit está disponible en: NgrokTunnel: "https://4e1e-35-185-9-112.ngrok-free.app" -> "http://localhost:8501"
¡Haz clic en el enlace de arriba para abrir tu dashboard!
